In [1]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("FPA_FOD_Plus.csv", low_memory=False)
df.head()

,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_PLUS_INCIDENT_JOIN_ID,ICS_209_PLUS_COMPLEX_JOIN_ID,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,LatLong_State,LatLong_County,NPL,Mang_Type,Mang_Name,Des_Tp,GAP_Sts,GAP_Prity,EVH,EVT,EVH_1km,EVT_1km,EVC,EVC_1km,NAME,MOD_NDVI_12m,MOD_EVI_12m,Land_Cover,Land_Cover_1km,rpms,rpms_1km,Population,Popo_1km,GACCAbbrev,GACC_PL,GACC_New fire,GACC_New LF,GACC_Uncont LF,GACC_Type 1 IMTs,GACC_Type 2 IMTs,GACC_NIMO Teams,GACC_Area Command Teams,GACC_Fire Use Teams,GDP,pr_Normal,tmmn_Normal,tmmx_Normal,rmin_Normal,rmax_Normal,sph_Normal,srad_Normal,fm100_Normal,fm1000_Normal,bi_Normal,vpd_Normal,erc_Normal,DF_PFS,AF_PFS,HDF_PFS,DSF_PFS,EBF_PFS,EALR_PFS,EBLR_PFS,EPLR_PFS,HBF_PFS,LLEF_PFS,LIF_PFS,LMI_PFS,MHVF_PFS,PM25F_PFS,HSEF,P100_PFS,P200_PFS,LPF_PFS,NPL_PFS,RMP_PFS,TSDF_PFS,TPF,TF_PFS,UF_PFS,WF_PFS,M_WTR,M_WKFC,M_CLT,M_ENY,M_TRN,M_HSG,M_PLN,M_HLTH,SM_C,SM_PFS,EPLRLI,EALRLI,EBLRLI,PM25LI,EBLI,DPMLI,TPLI,LPMHVLI,HBLI,RMPLI,SFLI,HWLI,WDLI,DLI,ALI,HDLI,LLELI,LILHSE,PLHSE,LMILHSE,ULHSE,EPL_ET,EAL_ET,EBL_ET,EB_ET,PM25_ET,DS_ET,TP_ET,LPP_ET,HB_ET,RMP_ET,NPL_ET,TSDF_ET,WD_ET,DB_ET,A_ET,HD_ET,LLE_ET,UN_ET,LISO_ET,POV_ET,LMI_ET,IA_LMI_ET,IA_UN_ET,IA_POV_ET,TC,CC,IAULHSE,IAPLHSE,IALMILHSE,IALMIL_87,IAPLHS_88,IAULHS_89,LHE,IALHE,IAHSEF,CA,NCA,CA_LT20,M_CLT_EOMI,M_ENY_EOMI,M_TRN_EOMI,M_HSG_EOMI,M_PLN_EOMI,M_WTR_EOMI,M_HLTH_102,M_WKFC_103,FPL200S,M_WKFC_105,M_EBSI,UI_EXP,THRHLD,No_FireStation_1.0km,No_FireStation_5.0km,No_FireStation_10.0km,No_FireStation_20.0km,FRG_1km,FRG,TRI_1km,TRI,Aspect_1km,Elevation_1km,Elevation,Slope_1km,Aspect,Slope,GHM,TPI,TPI_1km,TRACT,RPL_THEMES,RPL_THEME1,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,RPL_THEME2,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,RPL_THEME3,EPL_MINRTY,EPL_LIMENG,RPL_THEME4,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,road_county_dis,road_interstate_dis,road_common_name_dis,road_other_dis,road_state_dis,road_US_dis,Ecoregion_US_L4CODE,Ecoregion_US_L3CODE,Ecoregion_NA_L3CODE,Ecoregion_NA_L2CODE,Ecoregion_NA_L1CODE,SDI,Annual_etr,Annual_precipitation,Annual_tempreture,Aridity_index,Evacuation,pr,tmmn,tmmx,rmin,rmax,sph,vs,th,srad,etr,fm100,fm1000,bi,vpd,erc,pr_5D_mean,tmmn_5D_mean,tmmx_5D_mean,rmin_5D_mean,rmax_5D_mean,sph_5D_mean,vs_5D_mean,th_5D_mean,srad_5D_mean,etr_5D_mean,fm100_5D_mean,fm1000_5D_mean,bi_5D_mean,vpd_5D_mean,erc_5D_mean,pr_5D_min,pr_5D_max,tmmn_5D_max,tmmx_5D_max,rmin_5D_min,rmax_5D_min,sph_5D_min,vs_5D_max,th_5D_max,srad_5D_max,etr_5D_max,fm100_5D_min,fm1000_5D_min,bi_5D_max,vpd_5D_max,erc_5D_max,tmmn_Percentile,tmmx_Percentile,sph_Percentile,vs_Percentile,fm100_Percentile,bi_Percentile,vpd_Percentile,erc_Percentile,NDVI-1day,NDVI_min,NDVI_max,NDVI_mean,CheatGrass,ExoticAnnualGrass,Medusahead,PoaSecunda,geometry
0,300300629,HIWMO-OA2805,INTERAGCY,IA-HIWMO,ST/C&L,USHIHNLX,City and County of Honolulu,HICNTY,Honolulu Fire Dept,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007,2007-01-01,1,NaN,Missing data/not specified/undetermined,Missing data/not specified/undetermined,NaN,NaN,NaN,NaN,0.1,A,21.574287,-158.107040,MISSING/NOT SPECIFIED,HI,Oahu,15003.0,Honolulu County,Hawaii,Honolulu,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'0.62' '0.65' '0.52' '0.43' '0.45' '0.46' '0.4...,'0.3' '0.36' '0.25' '0.21' '0.23' '0.24' '0.27...,0.0,NaN,0,0.000000,0.0000,5.7139,RMCC,1.0,1.0,0,0,0.0,1.0,0.0,0.0,0.0,51481.170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.48,0.63,0.25,0.07,0.30,0.30,0.47,0.04,0.84,0.59,0.72,NaN,0.95,NaN,0.12,0.10,0.29,0.46,0.58,0.11,0.12,3884.0,0.41,0.48,0.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2302521 entries, 0 to 2302520
Columns: 308 entries, FOD_ID to geometry
dtypes: float64(239), int64(10), object(59)
memory usage: 5.3+ GB


In [6]:
target_states = ["AZ", "CA", "CO", "ID", "MT", "NV", "NM", "OR", "UT", "WA", "WY"]

df = df[df['STATE'].isin(target_states)]

In [7]:
# Create distance2road column by taking first non-null value from road distance columns
road_cols = ['road_county_dis', 'road_interstate_dis', 'road_common_name_dis', 
             'road_other_dis', 'road_state_dis', 'road_US_dis']

# Check which columns actually have data and their overlap
print("Non-null counts per column:")
print(df[road_cols].notna().sum())
print("\nSample of first few rows to see the pattern:")
print(df[road_cols].head(10))

# Take the minimum distance across all road types (nearest road of any kind)
df['Distance2road'] = df[road_cols].min(axis=1)

print("Non-null counts:")
print(f"distance2road (min): {df['Distance2road'].notna().sum()}")

Non-null counts per column:
road_county_dis          12049
road_interstate_dis        511
road_common_name_dis    490300
road_other_dis            4751
road_state_dis           10324
road_US_dis               1757
dtype: int64

Sample of first few rows to see the pattern:
    road_county_dis  road_interstate_dis  road_common_name_dis  \
2               NaN                  NaN                  43.0   
8               NaN                  NaN                  40.2   
14              NaN                  NaN                  43.8   
15              NaN                  NaN                  43.8   
16              NaN                  NaN                  41.0   
24              NaN                  NaN                   7.3   
28              NaN                  NaN                  39.3   
31              NaN                  NaN                  36.2   
54              NaN                  NaN                  39.8   
57              NaN                  NaN                  38.5   



In [8]:
target = "NWCG_GENERAL_CAUSE"  
vars = [
    # NEW
    "DISCOVERY_DOY", "FIRE_YEAR", "STATE",
    "FIPS_CODE", "Annual_etr", "Annual_precipitation", "Annual_tempreture", "pr", "tmmn", "vs",
    "fm100", "fm1000", "bi", "vpd", "erc", "Elevation_1km", "Aspect_1km",  "erc_Percentile", "Slope_1km", "TPI_1km",
    "EVC", "Evacuation", "SDI", "FRG", "No_FireStation_5.0km", "Mang_Name", "GAP_Sts", "GACC_PL", "GDP",
    "GHM", "NDVI-1day", "NPL", "Popo_1km", "RPL_THEMES", "RPL_THEME1", "RPL_THEME2", "RPL_THEME3", "RPL_THEME4", "Distance2road"]

In [9]:
keep_cols = vars + [target]
df_model = df[[c for c in keep_cols if c in df.columns]].copy()
df_model.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 752464 entries, 2 to 2302508
Data columns (total 40 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   DISCOVERY_DOY         752464 non-null  int64  
 1   FIRE_YEAR             752464 non-null  int64  
 2   STATE                 752464 non-null  object 
 3   FIPS_CODE             473290 non-null  float64
 4   Annual_etr            752464 non-null  int64  
 5   Annual_precipitation  752464 non-null  int64  
 6   Annual_tempreture     752464 non-null  float64
 7   pr                    752464 non-null  float64
 8   tmmn                  752464 non-null  float64
 9   vs                    752464 non-null  float64
 10  fm100                 752464 non-null  float64
 11  fm1000                752464 non-null  float64
 12  bi                    752464 non-null  float64
 13  vpd                   752464 non-null  float64
 14  erc                   752464 non-null  float64
 15  Elev

In [10]:
# Show all columns instead of truncated
pd.set_option('display.max_columns', None)
print(df_model.head())
df_model.info()

    DISCOVERY_DOY  FIRE_YEAR STATE  FIPS_CODE  Annual_etr  \
2               1       2007    CA     6053.0        1625   
8               1       2007    CA     6019.0        1819   
14              1       2007    CA     6089.0        2293   
15              1       2007    CA     6089.0        2293   
16              1       2007    CA     6079.0        2423   

    Annual_precipitation  Annual_tempreture   pr        tmmn   vs  fm100  \
2                    257              286.0  0.0  276.500000  6.8   14.7   
8                    383              290.0  0.0  273.200012  2.4   13.1   
14                   985              290.0  0.0  275.100006  3.6   14.8   
15                   985              290.0  0.0  275.100006  3.6   14.8   
16                   102              289.0  0.0  271.299988  3.3   16.3   

    fm1000    bi   vpd   erc  Elevation_1km  Aspect_1km erc_Percentile  \
2     19.4  38.0  0.47  27.0      85.913912  169.522268         70-90%   
8     17.2  26.0  0.47  34.0

In [11]:
print(df_model.isnull().sum().sort_values(ascending=False))

Evacuation              518445
GACC_PL                 380737
No_FireStation_5.0km    378768
FIPS_CODE               279174
Distance2road           232772
Popo_1km                200054
NDVI-1day                21558
RPL_THEME1                1260
RPL_THEME2                1260
RPL_THEME4                1260
RPL_THEME3                1260
RPL_THEMES                1260
erc_Percentile             535
GAP_Sts                     20
GDP                         20
Mang_Name                   20
pr                           0
Annual_tempreture            0
Annual_precipitation         0
Annual_etr                   0
STATE                        0
FIRE_YEAR                    0
DISCOVERY_DOY                0
tmmn                         0
FRG                          0
SDI                          0
EVC                          0
TPI_1km                      0
Aspect_1km                   0
Slope_1km                    0
erc                          0
Elevation_1km                0
fm1000  

In [12]:
# Convert erc_Percentile values to numeric
def convert_erc_percentile(val):
    if pd.isna(val):
        return np.nan
    val = val.strip()
    if val.startswith('<'):
        try:
            num = float(val[1:].replace('%',''))
            return num / 2
        except:
            return np.nan
    elif '-' in val:
        parts = val.replace('%', '').split('-')
        try:
            lower, upper = float(parts[0]), float(parts[1])
            return (lower + upper) / 2
        except:
            return np.nan
    else:
        try:
            return float(val.replace('%',''))
        except:
            return np.nan

# Apply transformations to df_model dataframe
def preprocess_dataframe(df_model):
   # Process erc_Percentile
    df_model['erc_Percentile'] = df_model['erc_Percentile'].apply(convert_erc_percentile)
    
    # Convert object columns that are potentially numeric strings
    obj_cols = df_model.select_dtypes(include='object').columns
    for col in obj_cols:
        if col in ['erc_Percentile']:
            # Already processed
            continue
        # Replace pd.NA with np.nan before conversion
        df_model[col] = df_model[col].replace({pd.NA: np.nan})
        try:
            df_model[col] = df_model[col].astype(float)
        except ValueError:
            # Leave as object if conversion fails
            pass

    return df_model

df_model = preprocess_dataframe(df_model)
df_model.head()

,DISCOVERY_DOY,FIRE_YEAR,STATE,FIPS_CODE,Annual_etr,Annual_precipitation,Annual_tempreture,pr,tmmn,vs,fm100,fm1000,bi,vpd,erc,Elevation_1km,Aspect_1km,erc_Percentile,Slope_1km,TPI_1km,EVC,Evacuation,SDI,FRG,No_FireStation_5.0km,Mang_Name,GAP_Sts,GACC_PL,GDP,GHM,NDVI-1day,NPL,Popo_1km,RPL_THEMES,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,Distance2road,NWCG_GENERAL_CAUSE
2,1,2007,CA,6053.0,1625,257,286.0,0.0,276.500000,6.8,14.7,19.4,38.0,0.47,27.0,85.913912,169.522268,80.0,11.784206,0.01,25.0,NaN,0.00,1,1.0,UNK,4.0,1.0,55688.996,0.42,NaN,1.0,1.1494,0.055,0.027,0.245,0.039,0.203,43.0,Misuse of fire by a minor
8,1,2007,CA,6019.0,1819,383,290.0,0.0,273.200012,2.4,13.1,17.2,26.0,0.47,34.0,470.351748,152.385082,60.0,8.974592,-0.04,25.0,NaN,0.26,4,NaN,UNK,4.0,1.0,55688.996,0.35,0.50,1.0,0.1652,0.525,0.719,0.499,0.302,0.405,40.2,Arson/incendiarism
14,1,2007,CA,6089.0,2293,985,290.0,0.0,275.100006,3.6,14.8,23.3,22.0,0.59,19.0,283.342485,121.181775,80.0,10.699408,-0.02,25.0,NaN,0.00,1,1.0,USFS,3.0,1.0,55688.996,0.16,0.42,1.0,0.0504,0.476,0.635,0.516,0.002,0.581,43.8,Misuse of fire by a minor
15,1,2007,CA,6089.0,2293,985,290.0,0.0,275.100006,3.6,14.8,23.3,22.0,0.59,19.0,283.342485,121.181775,80.0,10.699408,-0.02,25.0,NaN,0.00,1,1.0,USFS,3.0,1.0,55688.996,0.16,0.42,1.0,0.0504,0.476,0.635,0.516,0.002,0.581,43.8,Misuse of fire by a minor
16,1,2007,CA,6079.0,2423,102,289.0,0.0,271.299988,3.3,16.3,15.6,30.0,0.32,34.0,474.715385,-0.802098,80.0,-0.601166,-0.00,16.0,2.0,0.01,4,1.0,UNK,4.0,1.0,55688.996,0.18,0.16,1.0,0.0718,0.295,0.309,0.321,0.105,0.313,41.0,Debris and open burning


In [13]:
df_model['Evacuation'] = df['Evacuation'].fillna(0)

In [14]:
df_model['No_FireStation_5.0km'] = df['No_FireStation_5.0km'].fillna(0)

In [15]:
df_model['NDVI-1day'] = df['NDVI-1day'].fillna(0)

In [16]:
df_model['GACC_PL'] = df['GACC_PL'].fillna(0)

In [17]:
print(df_model.isnull().sum().sort_values(ascending=False))

FIPS_CODE               279174
Distance2road           232772
Popo_1km                200054
erc_Percentile          136378
RPL_THEME2                1260
RPL_THEME1                1260
RPL_THEMES                1260
RPL_THEME4                1260
RPL_THEME3                1260
Mang_Name                   20
GDP                         20
GAP_Sts                     20
STATE                        0
FIRE_YEAR                    0
DISCOVERY_DOY                0
Annual_etr                   0
pr                           0
Annual_tempreture            0
Annual_precipitation         0
tmmn                         0
bi                           0
vpd                          0
fm100                        0
vs                           0
FRG                          0
SDI                          0
Evacuation                   0
EVC                          0
TPI_1km                      0
Slope_1km                    0
Aspect_1km                   0
Elevation_1km                0
fm1000  

In [18]:
df_model = df_model.dropna()
print(f"Remaining rows after dropping incomplete cases: {df_model.shape[0]}")

Remaining rows after dropping incomplete cases: 231384


In [19]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 231384 entries, 2 to 2302506
Data columns (total 40 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   DISCOVERY_DOY         231384 non-null  int64  
 1   FIRE_YEAR             231384 non-null  int64  
 2   STATE                 231384 non-null  object 
 3   FIPS_CODE             231384 non-null  float64
 4   Annual_etr            231384 non-null  int64  
 5   Annual_precipitation  231384 non-null  int64  
 6   Annual_tempreture     231384 non-null  float64
 7   pr                    231384 non-null  float64
 8   tmmn                  231384 non-null  float64
 9   vs                    231384 non-null  float64
 10  fm100                 231384 non-null  float64
 11  fm1000                231384 non-null  float64
 12  bi                    231384 non-null  float64
 13  vpd                   231384 non-null  float64
 14  erc                   231384 non-null  float64
 15  Elev

In [20]:
df_model.to_csv('FPA-FOD_39attributes.csv')  